In [243]:
import re
import nltk

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nikita\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## 1. Представление и предобработка текстовых данных в виде последовательностей

1.1 Представьте первое предложение из строки `text` как последовательность из индексов слов, входящих в это предложение

In [4]:
text = 'Select your preferences and run the install command. Stable represents the most currently tested and supported version of PyTorch. Note that LibTorch is only available for C++'

In [10]:
sub = re.compile("[A-Za-z]+")
text_words = set(word for word in nltk.word_tokenize(text.lower()) if sub.search(word))
ind_to_word = dict(enumerate(text_words))
word_to_ind = {j:i for i,j in ind_to_word.items()}
word_to_ind

{'supported': 0,
 'c++': 1,
 'stable': 2,
 'most': 3,
 'note': 4,
 'of': 5,
 'select': 6,
 'currently': 7,
 'pytorch': 8,
 'version': 9,
 'your': 10,
 'only': 11,
 'and': 12,
 'that': 13,
 'run': 14,
 'command': 15,
 'is': 16,
 'represents': 17,
 'tested': 18,
 'the': 19,
 'install': 20,
 'for': 21,
 'libtorch': 22,
 'preferences': 23,
 'available': 24}

In [8]:
fs = 'Select your preferences and run the install command'
fs = fs.lower()
fs = nltk.word_tokenize(fs)

In [11]:
fs_vect = torch.zeros(len(fs))
for i,word in enumerate(fs):
    fs_vect[i] = word_to_ind[word]
fs_vect

tensor([ 6., 10., 23., 12., 14., 19., 20., 15.])

1.2 Представьте первое предложение из строки `text` как последовательность векторов, соответствующих индексам слов. Для представления индекса в виде вектора используйте унитарное кодирование. В результате должен получиться двумерный тензор размера `количество слов в предложении` x `количество уникальных слов`

In [ ]:
text = 'Select your preferences and run the install command. Stable represents the most currently tested and supported version of PyTorch. Note that LibTorch is only available for C++'

In [15]:
fs_matrx = torch.zeros((len(fs), len(word_to_ind)))
for i, word in enumerate(fs):
    fs_matrx[i, word_to_ind[word]] = 1
fs_matrx

tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.]])

1.3 Решите задачу 1.2, используя модуль `nn.Embedding`

In [89]:
fs_vect_ = fs_vect.type(torch.long)
fs_vect_

tensor([ 6, 10, 23, 12, 14, 19, 20, 15])

In [90]:
nn.Embedding(num_embeddings=len(text_words), embedding_dim=len(text_words))(fs_vect_)

tensor([[-0.5549, -1.3717,  0.8663,  0.3970,  1.7596,  0.1559,  1.6309,  0.9894,
          0.3076, -0.7276, -0.1996, -0.4460,  0.5722,  0.8552, -0.7268,  1.0791,
         -0.1829, -0.6409,  1.8228, -2.0906, -0.7622, -1.3581, -1.0597, -0.3444,
          0.7982],
        [-0.1200,  2.0094, -1.3228,  0.0124,  1.7570,  0.5700, -0.2939, -1.2750,
          0.9414, -0.8880,  0.4092, -0.8743, -0.4598,  0.0810,  1.1493,  1.2512,
         -2.5529,  1.4449, -1.3563,  1.8004, -0.8388,  1.0175,  0.9995, -0.8984,
          1.7567],
        [ 0.1817,  0.2718,  0.1625,  0.2713,  0.5173, -0.7460, -1.3254,  1.2003,
          1.2920, -1.0720, -0.1621,  2.1803, -1.6398, -0.7761, -0.0253, -0.3782,
         -0.7283,  1.2035, -0.5046, -0.8360,  0.0859, -2.2821,  0.7486,  0.2217,
         -1.1833],
        [ 0.6319, -1.6133,  1.3906,  0.3368,  0.0168, -1.3093,  1.1511,  0.6371,
          0.1609, -0.0249, -0.2789, -0.5414,  1.0936, -0.4611,  0.7372,  0.3766,
         -0.4738, -1.1117, -0.7908,  1.1568, -0.7415

## 2. Классификация фамилий по национальности (ConvNet)

Датасет: https://disk.yandex.ru/d/owHew8hzPc7X9Q?w=1

2.1 Считать файл `surnames/surnames.csv`. 

In [150]:
df = pd.read_csv("./data/surnames.csv")

2.2 Закодировать национальности числами, начиная с 0.

In [151]:
class_encoder = LabelEncoder()
df['nationality'] = class_encoder.fit_transform(df['nationality'])

2.3 Разбить датасет на обучающую и тестовую выборку

In [152]:
X = df['surname'].str.lower().str.strip()
y = df['nationality']
n_classes = y.nunique()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

2.4 Реализовать класс `Vocab` (токен = __символ__)
  * добавьте в словарь специальный токен `<PAD>` с индексом 0
  * при создании словаря сохраните длину самой длинной последовательности из набора данных в виде атрибута `max_seq_len`

In [154]:
class Vocab:
    def __init__(self, data):
        self.max_seq_len = data.str.len().max()
        tokens = set()
        for item in data:
            tokens.update(item)
        tokens = list(tokens)
        tokens.insert(0, "<PAD>")
        self.idx_to_token = dict(enumerate(tokens))
        self.token_to_idx = {token: idx for idx, token in self.idx_to_token.items()}
        self.vocab_len = len(self.idx_to_token)

2.5 Реализовать класс `SurnamesDataset`
  * метод `__getitem__` возвращает пару: <последовательность индексов токенов (см. 1.1 ), номер класса> 
  * длина каждой такой последовательности должна быть одинаковой и равной `vocab.max_seq_len`. Чтобы добиться этого, дополните последовательность справа индексом токена `<PAD>` до нужной длины


In [155]:
class SurnamesDataset(Dataset):
    def __init__(self, X, y, vocab: Vocab):
        self.X = X
        self.y = y
        self.vocab = vocab

    def vectorize(self, surname):
        '''Генерирует представление фамилии surname в при помощи бинарного кодирования (см. 1.2)'''
        surname_t = torch.zeros(self.vocab.max_seq_len).type(torch.long)
        for i, token in enumerate(surname):
            surname_t[i] = self.vocab.token_to_idx[token]
        return surname_t

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.vectorize(self.X.iloc[idx]), self.y.iloc[idx]

2.6. Обучить классификатор.
  
  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding`. Рассмотрите два варианта: 
    - когда токен представляется в виде унитарного вектора и модуль `nn.Embedding` не обучается
    - когда токен представляется в виде вектора небольшой размерности (меньше, чем размер словаря) и модуль `nn.Embedding` обучается

  * Используйте одномерные свертки и пулинг (`nn.Conv1d`, `nn.MaxPool1d`)
    - обратите внимание, что `nn.Conv1d` ожидает на вход трехмерный тензор размерности `(batch, embedding_dim, seq_len)`


In [245]:
def hard_training(model, criterion, optimizer, n_epochs=51):
    for epoch in range(n_epochs):
        epoch_loss = 0
        y_pred = torch.empty(0)
        y_true = torch.empty(0)
        for X_batch, y_batch in train_loader:
            predictions = model(X_batch)
            loss = criterion(predictions, y_batch.type(torch.long))
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            epoch_loss += loss.item()
            with torch.no_grad():
                y_true = torch.cat((y_true, y_batch))
                y_pred = torch.cat((y_pred, predictions.argmax(dim=1).cpu().detach()))
        trin_acc = accuracy_score(y_true, y_pred).item()
        
        with torch.no_grad():
            val_loss = 0
            y_pred = torch.empty(0)
            y_true = torch.empty(0)
            for X_batch, y_batch in test_loader:
                predictions = model(X_batch)
                loss = criterion(predictions, y_batch.type(torch.long)).item()
                y_true = torch.cat((y_true, y_batch))
                y_pred = torch.cat((y_pred, predictions.argmax(dim=1).cpu().detach()))
                val_loss += loss
            val_acc = accuracy_score(y_true, y_pred).item()
            if epoch % 5 == 0:
                print(f'#{epoch} Training loss: {epoch_loss / len(train_loader):.4f}\
 training_acc: {trin_acc:.4f} val_loss: {val_loss / len(test_loader):.4f} val_acc: {val_acc:.4f}')
                
def GloP_boB(model):
    y_pred = torch.empty(0)
    y_ = torch.empty(0)

    for X_batch, y_batch in test_loader:

        predictions = model(X_batch).argmax(dim=1).cpu().detach()
        y_pred = torch.cat((y_pred, predictions))
        y_ = torch.cat((y_, y_batch))

    print(classification_report(y_, y_pred, zero_division=0))

In [210]:
vocab = Vocab(X)
train_dataset = SurnamesDataset(X_train, y_train, vocab)
test_dataset = SurnamesDataset(X_test, y_test, vocab)

In [211]:
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [233]:
class MyOwnClassifier(nn.Module):
    def __init__(self, num_embeddings, embedding_dim):
        super(MyOwnClassifier, self).__init__()
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)
        self.l1 = nn.Sequential(
            nn.Conv1d(embedding_dim, 64, kernel_size=5),
            nn.BatchNorm1d(64),
            nn.MaxPool1d(2),
        )
#         self.l2 = nn.Sequential(
#             nn.Conv1d(64, 128, kernel_size=3),
#             nn.BatchNorm1d(128),
#             nn.MaxPool1d(2),
#         )
        self.fc1 = nn.Sequential(
            nn.Dropout(),
            nn.Linear(3*128, 1024),
            nn.ReLU(),
            
        )
        self.fc2 = nn.Sequential(
            nn.Dropout(),
            nn.Linear(1024, 18),
            nn.Softmax(dim=1),
        )

    def forward(self, x, training=False):
        x = self.embedding(x).transpose(1,2)
        x = self.l1(x)
#         x = self.l2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [236]:
model = MyOwnClassifier(num_embeddings=vocab.vocab_len, embedding_dim=1)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [237]:
hard_training(model, criterion, optimizer)

#0 Training loss: 2.6574 training_acc: 0.3266 val_loss: 2.6476 val_acc: 0.3347
#5 Training loss: 2.6069 training_acc: 0.3749 val_loss: 2.6026 val_acc: 0.3780
#10 Training loss: 2.5764 training_acc: 0.4041 val_loss: 2.5774 val_acc: 0.4053
#15 Training loss: 2.5674 training_acc: 0.4128 val_loss: 2.5733 val_acc: 0.4080
#20 Training loss: 2.5451 training_acc: 0.4352 val_loss: 2.5413 val_acc: 0.4399
#25 Training loss: 2.5310 training_acc: 0.4512 val_loss: 2.5237 val_acc: 0.4577
#30 Training loss: 2.5213 training_acc: 0.4604 val_loss: 2.5195 val_acc: 0.4577
#35 Training loss: 2.5085 training_acc: 0.4735 val_loss: 2.5087 val_acc: 0.4672
#40 Training loss: 2.5026 training_acc: 0.4788 val_loss: 2.4957 val_acc: 0.4827
#45 Training loss: 2.4913 training_acc: 0.4909 val_loss: 2.5044 val_acc: 0.4740
#50 Training loss: 2.4874 training_acc: 0.4950 val_loss: 2.4926 val_acc: 0.4882


In [246]:
GloP_boB(model)

              precision    recall  f1-score   support

         0.0       0.52      0.79      0.62       346
         1.0       0.00      0.00      0.00        36
         2.0       0.00      0.00      0.00        81
         3.0       0.00      0.00      0.00        49
         4.0       0.41      0.80      0.54       567
         5.0       0.00      0.00      0.00        36
         6.0       0.00      0.00      0.00       118
         7.0       0.00      0.00      0.00        32
         8.0       0.00      0.00      0.00        41
         9.0       0.00      0.00      0.00       108
        10.0       0.47      0.35      0.40       161
        11.0       0.00      0.00      0.00        15
        12.0       0.00      0.00      0.00        25
        13.0       0.00      0.00      0.00        14
        14.0       0.64      0.58      0.61       482
        15.0       0.00      0.00      0.00        13
        16.0       0.00      0.00      0.00        57
        17.0       0.00    

In [240]:
model2 = MyOwnClassifier(num_embeddings=vocab.vocab_len, embedding_dim=32)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model2.parameters(), lr=0.001)

In [241]:
hard_training(model2, criterion, optimizer, n_epochs=101)

#0 Training loss: 2.5646 training_acc: 0.4235 val_loss: 2.4824 val_acc: 0.4954
#5 Training loss: 2.4261 training_acc: 0.5556 val_loss: 2.4259 val_acc: 0.5524
#10 Training loss: 2.4004 training_acc: 0.5814 val_loss: 2.4085 val_acc: 0.5669
#15 Training loss: 2.3654 training_acc: 0.6169 val_loss: 2.3710 val_acc: 0.6088
#20 Training loss: 2.3509 training_acc: 0.6302 val_loss: 2.3639 val_acc: 0.6189
#25 Training loss: 2.3232 training_acc: 0.6601 val_loss: 2.3448 val_acc: 0.6362
#30 Training loss: 2.3097 training_acc: 0.6717 val_loss: 2.3207 val_acc: 0.6585
#35 Training loss: 2.3022 training_acc: 0.6793 val_loss: 2.3244 val_acc: 0.6507
#40 Training loss: 2.2963 training_acc: 0.6851 val_loss: 2.3195 val_acc: 0.6594
#45 Training loss: 2.2878 training_acc: 0.6943 val_loss: 2.3330 val_acc: 0.6448
#50 Training loss: 2.2849 training_acc: 0.6967 val_loss: 2.3102 val_acc: 0.6648
#55 Training loss: 2.2819 training_acc: 0.6997 val_loss: 2.3161 val_acc: 0.6644
#60 Training loss: 2.2786 training_acc: 0.

In [247]:
GloP_boB(model2)

              precision    recall  f1-score   support

         0.0       0.88      0.91      0.90       346
         1.0       0.48      0.86      0.61        36
         2.0       0.00      0.00      0.00        81
         3.0       0.00      0.00      0.00        49
         4.0       0.58      0.92      0.71       567
         5.0       0.00      0.00      0.00        36
         6.0       0.44      0.16      0.24       118
         7.0       0.00      0.00      0.00        32
         8.0       0.00      0.00      0.00        41
         9.0       0.49      0.74      0.59       108
        10.0       0.77      0.85      0.81       161
        11.0       0.00      0.00      0.00        15
        12.0       0.00      0.00      0.00        25
        13.0       0.00      0.00      0.00        14
        14.0       0.80      0.79      0.80       482
        15.0       0.00      0.00      0.00        13
        16.0       0.00      0.00      0.00        57
        17.0       0.00    

2.7 Измерить точность на тестовой выборке. Проверить работоспособность модели: прогнать несколько фамилий студентов группы через модели и проверить результат. Для каждой фамилии выводить 3 наиболее вероятных предсказания.

In [266]:
def test_surname(surname):
    x = train_dataset.vectorize(surname).unsqueeze(0)
    variety, predictions = model(x).topk(k=3, dim=1)
    variety = variety.cpu().detach().squeeze()
    pred_ = class_encoder.inverse_transform(predictions.cpu().detach().squeeze())
    out_ = ", ".join([f"{nat}:{frac:.2f}" for nat, frac in zip(pred_, variety)])
    print(f"{surname} --- {out_}")

In [268]:
for surname in ("bazoyan", "gorshenev", "strizhov", "petukhov", "trubadurov", "atayanc"):
    test_surname(surname)

bazoyan --- Russian:1.00, Japanese:0.00, English:0.00
gorshenev --- Russian:0.99, English:0.01, Greek:0.00
strizhov --- Russian:1.00, English:0.00, Greek:0.00
petukhov --- Russian:1.00, English:0.00, Greek:0.00
trubadurov --- Russian:1.00, Japanese:0.00, German:0.00
atayanc --- English:1.00, Arabic:0.00, Greek:0.00


## 3. Классификация обзоров на фильмы (ConvNet)

Датасет: https://disk.yandex.ru/d/tdinpb0nN_Dsrg

2.1 Создайте набор данных на основе файлов polarity/positive_reviews.csv (положительные отзывы) и polarity/negative_reviews.csv (отрицательные отзывы). Разбейте на обучающую и тестовую выборку.
  * токен = __слово__
  * данные для обучения в датасете представляются в виде последовательности индексов токенов
  * словарь создается на основе _только_ обучающей выборки. Для корректной обработки ситуаций, когда в тестовой выборке встретится токен, который не хранится в словаре, добавьте в словарь специальный токен `<UNK>`
  * добавьте предобработку текста

In [327]:
positive = pd.read_csv("./data/positive_reviews.txt", sep='%-%', header=None, engine="python")
positive["type"] = "positive"
negative = pd.read_csv("./data/negative_reviews.txt", sep='%-%', header=None, engine="python")
negative["type"] = "negative"
df = pd.concat((positive, negative), ignore_index=True)
df.columns = ["review", "type"]
df.head(2)

,review,type
0,"simplistic , silly and tedious .",positive
1,"it's so laddish and juvenile , only teenage bo...",positive


In [328]:
class_encoder_2 = LabelEncoder()
df.type = class_encoder_2.fit_transform(df.type)

In [329]:
stopwords = nltk.corpus.stopwords.words('english')
lemmatizer = nltk.stem.WordNetLemmatizer()
puttern = re.compile("^[a-z]+$")
def preprocess(text):
    text = text.lower().strip()
    words = [lemmatizer.lemmatize(word) for word in nltk.word_tokenize(text) if (puttern.search(word)) and (word not in stopwords)]
    return words

In [330]:
df.review = df.review.apply(preprocess)

In [332]:
X = df.review
y = df.type
n_classes = y.nunique()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [336]:
class Vocab:
    def __init__(self, data):
        self.max_seq_len = data.apply(lambda x: len(x)).max()
        tokens = set()
        for item in data:
            tokens.update(item)
        tokens = list(tokens)
        tokens.insert(0, "<PAD>")
        tokens.insert(1, "<UNK>")
        self.idx_to_token = dict(enumerate(tokens))
        self.token_to_idx = {token: idx for idx, token in self.idx_to_token.items()}
        self.vocab_len = len(self.idx_to_token)

In [357]:
class ReviewsDataset(Dataset):
    def __init__(self, X, y, vocab: Vocab):
        self.X = X
        self.y = y
        self.vocab = vocab

    def vectorize(self, review):
        '''Генерирует представление отзыва'''
        review_t = torch.zeros(self.vocab.max_seq_len).type(torch.long)
        for i, token in enumerate(review):
            try:
                review_t[i] = self.vocab.token_to_idx[token]
            except KeyError as ke:
                review_t[i] = self.vocab.token_to_idx["<UNK>"]
        return review_t

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.vectorize(self.X.iloc[idx]), self.y.iloc[idx]

2.2. Обучите классификатор.
  
  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding` 
    - подберите адекватную размерность вектора эмбеддинга: 
    - модуль `nn.Embedding` обучается

  * Используйте одномерные свертки и пулинг (`nn.Conv1d`, `nn.MaxPool1d`)
    - обратите внимание, что `nn.Conv1d` ожидает на вход трехмерный тензор размерности `(batch, embedding_dim, seq_len)`

In [381]:
vocab = Vocab(X_train)
train_dataset = ReviewsDataset(X_train, y_train, vocab)
test_dataset = ReviewsDataset(X_test, y_test, vocab)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [479]:
class MyOwnClassifier(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, dropout_level=0.5):
        super(MyOwnClassifier, self).__init__()
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)
        self.l1 = nn.Sequential(
            nn.Conv1d(embedding_dim, 256, kernel_size=5),
            nn.BatchNorm1d(256),
            nn.MaxPool1d(2),
        )
        self.l2 = nn.Sequential(
            nn.Conv1d(256, 512, kernel_size=3),
            nn.BatchNorm1d(512),
            nn.MaxPool1d(2),
        )
#         self.l3 = nn.Sequential(
#             nn.Conv1d(256, 512, kernel_size=3),
#             nn.BatchNorm1d(512),
#             nn.MaxPool1d(2),
#         )
        self.fc1 = nn.Sequential(
            nn.Dropout(p=dropout_level),
            nn.Linear(6*512, 1024),
#             nn.Softmax(dim=1),
            nn.ReLU(),
            
        )
#         self.fc2 = nn.Sequential(
#             nn.Dropout(p=dropout_level),
#             nn.Linear(1024, 256),
# #             nn.Softmax(dim=1),
#             nn.ReLU(),  
#         )
        self.fc3 = nn.Sequential(
            nn.Dropout(p=dropout_level),
            nn.Linear(1024, n_classes),
            nn.Softmax(dim=1),
        )
        
    def forward(self, x, training=False):
        x = self.embedding(x).transpose(1,2)
        x = self.l1(x)
        x = self.l2(x)
#         x = self.l3(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
#         x = self.fc2(x)
        x = self.fc3(x)
        return x

In [480]:
model = MyOwnClassifier(num_embeddings=vocab.vocab_len, embedding_dim=128, dropout_level=0.9)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [481]:
hard_training(model, criterion, optimizer, n_epochs=101)

#0 Training loss: 0.7622 training_acc: 0.4992 val_loss: 0.7590 val_acc: 0.4965
#5 Training loss: 0.7501 training_acc: 0.5083 val_loss: 0.7603 val_acc: 0.4876
#10 Training loss: 0.7292 training_acc: 0.5186 val_loss: 0.7365 val_acc: 0.5152
#15 Training loss: 0.7196 training_acc: 0.5256 val_loss: 0.7289 val_acc: 0.5077
#20 Training loss: 0.6861 training_acc: 0.5762 val_loss: 0.7172 val_acc: 0.5401
#25 Training loss: 0.6463 training_acc: 0.6349 val_loss: 0.7124 val_acc: 0.5504
#30 Training loss: 0.6093 training_acc: 0.6783 val_loss: 0.7082 val_acc: 0.5640
#35 Training loss: 0.5614 training_acc: 0.7358 val_loss: 0.7045 val_acc: 0.5799
#40 Training loss: 0.5113 training_acc: 0.7933 val_loss: 0.7027 val_acc: 0.5813
#45 Training loss: 0.4677 training_acc: 0.8398 val_loss: 0.7031 val_acc: 0.5879
#50 Training loss: 0.4386 training_acc: 0.8695 val_loss: 0.7032 val_acc: 0.5813
#55 Training loss: 0.4050 training_acc: 0.9050 val_loss: 0.6941 val_acc: 0.6001
#60 Training loss: 0.3856 training_acc: 0.

In [482]:
GloP_boB(model)

              precision    recall  f1-score   support

         0.0       0.62      0.47      0.54      1067
         1.0       0.58      0.72      0.64      1066

    accuracy                           0.59      2133
   macro avg       0.60      0.59      0.59      2133
weighted avg       0.60      0.59      0.59      2133



2.7 Измерить точность на тестовой выборке. Проверить работоспособность модели: придумать небольшой отзыв, прогнать его через модель и вывести номер предсказанного класса (сделать это для явно позитивного и явно негативного отзыва)
* Целевое значение accuracy на валидации - 70+%

In [502]:
def test_review(review):
    x = train_dataset.vectorize(review).unsqueeze(0)
    variety, predictions = model(x).topk(k=1, dim=1)
    variety = variety.cpu().detach().view(-1)
    pred_ = predictions.cpu().detach().view(-1)
    pred_ = class_encoder_2.inverse_transform(pred_)
    out_ = ", ".join([f"{nat}:{frac:.2f}" for nat, frac in zip(pred_, variety)])
    print(f"{review} --- {out_}")

In [511]:
reviews_ = [
    "The last time I had lunch here, I really liked the soup. Excellent. I'll come again.",
    "It feels like I dined on frogs. I will not visit this institution again."
]

In [512]:
for review in reviews_:
     test_review(preprocess(review))

['last', 'time', 'lunch', 'really', 'liked', 'soup', 'excellent', 'come'] --- negative:1.00
['feel', 'like', 'dined', 'frog', 'visit', 'institution'] --- positive:1.00


In [513]:
#лол все наоборот